In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
dataset = "google_reviews"
seeds = [4, 170, 340, 870, 5999]

In [3]:
item_info = pd.read_csv(f"dataset/{dataset}/{dataset}.item", sep="\t", engine="python", index_col=0, header=0)
with open(f"dataset/remappings/{dataset}.json") as f:
    remappings = json.load(f)

item_labels = item_info.loc[item_info["total ratings (%)"] > 0]["popular item"]
item_labels = pd.Series(item_labels.values, index=[remappings["item_id"][str(i)] for i in item_labels.index])

In [4]:
item_avgs = {item_idx: [] for item_idx in item_labels.index}

for seed in seeds:
    with open(f"scores_{dataset}_{seed}.json") as f:
        d = json.load(f)
    for item_idx in sorted(item_labels.index):
        item_scores = np.array([d[user_idx][item_idx] for user_idx in d.keys() if d[user_idx][item_idx] != -np.inf])
        if len(item_scores) > 0:
            item_avgs[item_idx].append(item_scores.mean())

item_stds = {item_idx: np.array(item_avgs[item_idx]).std() for item_idx in item_labels.index}

In [5]:
with open(f"uncertainty_{dataset}_std.json", "w") as f:
    json.dump(item_stds, f)